#INTRODUCTION

This notebook is a part of our final project for class CS221.O11 of University of information and technology

Our group:
- Lê Tuấn Vũ (21521679)
- Trần Nhật Phong (21521276)
- Nguyễn Nhật Minh (21521135)

This notebook aim to remove noise and other unnecessary noise from data to help the model learn only the important thing from the dataset.

Further this notebook also demonstrate our skill on analysing and processing data.



Thing we planning to do with progress:
- Remove emotion icon (DONE)
- Lowercase all character (DONE)
- Remove punctuation (DONE)
- Segmentation (DONE)
- Remove shorthand/teencode (*sử dụng spelling checking ý tưởng là từ dùng nhiều và có trong data*) (DONE)
- Spelling correction (DONE)
- Remove stopword (DONE)
- Train test split (80/20) (DONE)


# CONTENT

In [ ]:
from google.colab import drive
drive.mount("/content/drive")


Mounted at /content/drive


### Install Dependencies

In [ ]:
!pip install autocorrect
!pip install emot --upgrade
!pip install emoji
!pip install underthesea

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 622.8/622.8 kB 8.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for autocorrect: filename=autocorrect-2.6.1-py3-none-any.whl size=622364 sha256=ede6aa002102ea2380e2a35d1a782fa67382fbe1e8c4daf5e58aef70a17108f7
  Stored in directory: /root/.cache/pip/wheels/b5/7b/6d/b76b29ce11ff8e2521c8c7dd0e5bfee4fb1789d76193124343
Successfully built autocorrect
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.5/61.5 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 457.9/457.9 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.9/20.9 MB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 66.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 657.8/657.8 kB 52.2 MB/s eta 0:00:00


In [ ]:
!pip install py_vncorenlp

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 7.5 MB/s eta 0:00:00
  Created wheel for py_vncorenlp: filename=py_vncorenlp-0.1.4-py3-none-any.whl size=4307 sha256=710e6259f85c9d608b41fcd290c1de0c195242c0a8476e9798713461b949e0a3
  Stored in directory: /root/.cache/pip/wheels/d5/d9/bf/62632cdb007c702a0664091e92a0bb1f18a2fcecbe962d9827
Successfully built py_vncorenlp


In [ ]:
import os
import subprocess
import nltk
import pandas as pd
import re
import string
import emoji
import emot

from transformers import AutoModel, AutoTokenizer
from nltk.corpus import stopwords
from nltk import word_tokenize
from autocorrect import Speller

### Load data

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base-v2")
vocab = tokenizer.get_vocab()
not_in_vocab = {}

In [ ]:
ori_data = pd.read_csv("/content/drive/MyDrive/NLP Final Project/data.csv")

In [ ]:
data = pd.read_csv("/content/drive/MyDrive/NLP Final Project/data.csv")

In [ ]:
data.rename(columns={'start': 'star'}, inplace=True)

In [ ]:
data.head()

,content,label,star
0,Áo bao đẹp ạ!,POS,5
1,Tuyệt vời,POS,5
2,2day ao khong giong trong,NEG,1
3,"Mùi thơm,bôi lên da mềm da",POS,5
4,"Vải đẹp, dày dặn",POS,5


### Lowercase all characters

In [ ]:
data['content'] = data['content'].str.lower()

In [ ]:
data['content']

0                                            áo bao đẹp ạ!
1                                                tuyệt vời
2                                2day ao khong giong trong
3                               mùi thơm,bôi lên da mềm da
4                                         vải đẹp, dày dặn
                               ...                        
31455                                      không đáng tiền
31456                                         quần rất đẹp
31457                               hàng đẹp đúng giá tiền
31458                                      chất vải khá ổn
31459    áo rất ok nhé , vải mịn , len cao cổ này phối ...
Name: content, Length: 31460, dtype: object

### Drop NaN

In [ ]:
data = data.dropna(axis=0)
data

,content,label,star
0,áo bao đẹp ạ!,POS,5
1,tuyệt vời,POS,5
2,2day ao khong giong trong,NEG,1
3,"mùi thơm,bôi lên da mềm da",POS,5
4,"vải đẹp, dày dặn",POS,5
...,...,...,...
31455,không đáng tiền,NEG,1
31456,quần rất đẹp,POS,5
31457,hàng đẹp đúng giá tiền,POS,5
31458,chất vải khá ổn,POS,4


### Remove emoji and punctuation

In [ ]:
data['content_emoji'] = data['content'].apply(lambda s: emoji.replace_emoji(s, ' '))
data

,content,label,star,content_emoji
0,áo bao đẹp ạ!,POS,5,áo bao đẹp ạ!
1,tuyệt vời,POS,5,tuyệt vời
2,2day ao khong giong trong,NEG,1,2day ao khong giong trong
3,"mùi thơm,bôi lên da mềm da",POS,5,"mùi thơm,bôi lên da mềm da"
4,"vải đẹp, dày dặn",POS,5,"vải đẹp, dày dặn"
...,...,...,...,...
31455,không đáng tiền,NEG,1,không đáng tiền
31456,quần rất đẹp,POS,5,quần rất đẹp
31457,hàng đẹp đúng giá tiền,POS,5,hàng đẹp đúng giá tiền
31458,chất vải khá ổn,POS,4,chất vải khá ổn


In [ ]:
n = 0
for i in range(len(data)):
  try:
    if data['content'][i] != data['content_emoji'][i]:
      n+=1
  except:
    pass
print(n)

0


In [ ]:
data['content'] = data['content_emoji']
data.drop(columns = 'content_emoji', inplace = True)

In [ ]:
punct = '!"#$%&\'()*+,-./:;<=>?@[\\]^_`{}~'   # `|` is not present here
transtab = str.maketrans(dict.fromkeys(punct,' '))

data['content'] = '|'.join(data['content'].tolist()).translate(transtab).split('|')

In [ ]:
data['content'] = data['content'].str.strip()

In [ ]:
import numpy as np
data['content'].replace('', np.nan, inplace=True)

In [ ]:
data = data.dropna(axis=0)

###Segmentation

In [ ]:
import py_vncorenlp

In [ ]:
# Automatically download VnCoreNLP components from the original repository
# and save them in some local machine folder
py_vncorenlp.download_model(save_dir='/content/')

# Load the word and sentence segmentation component
rdrsegmenter = py_vncorenlp.VnCoreNLP(annotators=["wseg"], save_dir='/content/')

In [ ]:
data['content'] = data['content'].apply(lambda s: rdrsegmenter.word_segment(s)[0])

<ipython-input-23-2e0b543874e5>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['content'] = data['content'].apply(lambda s: rdrsegmenter.word_segment(s)[0])


In [ ]:
r_dict={}

In [ ]:
def find_not_in_vocab(save_dict = False):
  not_in_vocab = {}
  for sent in data['content']:
    if len(sent):
      for word in sent.split():
        if word not in vocab.keys():
          if word not in not_in_vocab.keys():
            not_in_vocab[word] = 1
          else:
            not_in_vocab[word] = not_in_vocab[word] + 1
  not_in_vocab = dict(sorted(not_in_vocab.items(), key=lambda item: item[1], reverse=True))
  if save_dict:
    with open('/content/drive/MyDrive/NLP Final Project/not_in_vocab.txt', 'w') as f:
      for word in not_in_vocab:
          f.write(f'{word}:{not_in_vocab[word]}\n')

In [ ]:
# Find word not in vocabulary
# find_not_in_vocab()

In [ ]:
with open('/content/drive/MyDrive/NLP Final Project/not_in_vocab.txt', 'r') as f:
  for line in f:
    line_compo = line.rstrip().split(':')
    r_dict[line_compo[0]]= line_compo[1]

In [ ]:
len(r_dict)

149

In [ ]:
data['spell_corrected'] = data['content'].apply(lambda s: ' '.join([r_dict.get(i, i) for i in s.split()]))

<ipython-input-29-06ff5e619fd4>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['spell_corrected'] = data['content'].apply(lambda s: ' '.join([r_dict.get(i, i) for i in s.split()]))


In [ ]:
data['content_no_num'] = data['spell_corrected'].str.replace('\d+', '')

<ipython-input-30-045139434316>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  data['content_no_num'] = data['spell_corrected'].str.replace('\d+', '')
<ipython-input-30-045139434316>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['content_no_num'] = data['spell_corrected'].str.replace('\d+', '')


In [ ]:
data

,content,label,star,spell_corrected,content_no_num
0,áo bao đẹp ạ,POS,5,áo bao đẹp ạ,áo bao đẹp ạ
1,tuyệt_vời,POS,5,tuyệt_vời,tuyệt_vời
2,2day ao khong giong trong,NEG,1,2day ao không giống trong,day ao không giống trong
3,mùi thơm bôi lên da mềm da,POS,5,mùi thơm bôi lên da mềm da,mùi thơm bôi lên da mềm da
4,vải đẹp dày_dặn,POS,5,vải đẹp dày_dặn,vải đẹp dày_dặn
...,...,...,...,...,...
31455,không đáng tiền,NEG,1,không đáng tiền,không đáng tiền
31456,quần rất đẹp,POS,5,quần rất đẹp,quần rất đẹp
31457,hàng đẹp đúng giá tiền,POS,5,hàng đẹp đúng giá tiền,hàng đẹp đúng giá tiền
31458,chất vải khá ổn,POS,4,chất vải khá ổn,chất vải khá ổn


### Load stop-words

In [ ]:
stopwords = []

with open("/content/drive/MyDrive/NLP Final Project/vietnamese.txt", "r") as f:
    stopwords = f.read().splitlines()

In [ ]:
stopwords = [word.replace(' ', '_') for word in stopwords]

In [ ]:
stopwords[:10]

['a_lô',
 'a_ha',
 'ai',
 'ai_ai',
 'ai_nấy',
 'ai_đó',
 'alô',
 'amen',
 'anh',
 'anh_ấy']

In [ ]:
data['no_stopwords'] = data['spell_corrected'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stopwords)]))

<ipython-input-36-291f4bb3576a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['no_stopwords'] = data['spell_corrected'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stopwords)]))


In [ ]:
data['fully_process'] = data['content_no_num'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stopwords)]))

<ipython-input-37-446333f3524b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['fully_process'] = data['content_no_num'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stopwords)]))


In [ ]:
data

,content,label,star,spell_corrected,content_no_num,no_stopwords,fully_process
0,áo bao đẹp ạ,POS,5,áo bao đẹp ạ,áo bao đẹp ạ,áo bao đẹp,áo bao đẹp
1,tuyệt_vời,POS,5,tuyệt_vời,tuyệt_vời,tuyệt_vời,tuyệt_vời
2,2day ao khong giong trong,NEG,1,2day ao không giống trong,day ao không giống trong,2day ao không giống trong,day ao không giống trong
3,mùi thơm bôi lên da mềm da,POS,5,mùi thơm bôi lên da mềm da,mùi thơm bôi lên da mềm da,mùi thơm bôi da mềm da,mùi thơm bôi da mềm da
4,vải đẹp dày_dặn,POS,5,vải đẹp dày_dặn,vải đẹp dày_dặn,vải đẹp dày_dặn,vải đẹp dày_dặn
...,...,...,...,...,...,...,...
31455,không đáng tiền,NEG,1,không đáng tiền,không đáng tiền,không đáng tiền,không đáng tiền
31456,quần rất đẹp,POS,5,quần rất đẹp,quần rất đẹp,quần đẹp,quần đẹp
31457,hàng đẹp đúng giá tiền,POS,5,hàng đẹp đúng giá tiền,hàng đẹp đúng giá tiền,hàng đẹp đúng giá tiền,hàng đẹp đúng giá tiền
31458,chất vải khá ổn,POS,4,chất vải khá ổn,chất vải khá ổn,chất vải ổn,chất vải ổn


In [ ]:
data.to_csv('/content/drive/MyDrive/NLP Final Project/preprocessed.csv')

### Train test split

In [ ]:
from sklearn.model_selection import train_test_split


y = data['label']
drop_list = ['content', 'spell_corrected', 'content_no_num', 'no_stopwords']
x = data.copy()
for td in drop_list:
  x = x.drop([td], axis=1,inplace=False)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state = 42)

In [ ]:
df_train = x_train.copy()
df_train['label'] = y_train

df_test = x_test.copy()
df_test['label'] = y_test



In [ ]:
df_train

,label,star,fully_process
11052,POS,4,đóng_gói sản_phẩm đẹp shop
7116,POS,4,chấm có chút_xíu r tán
27433,POS,5,áo blue k đẹp đấy shop
27034,POS,4,chất_lượng sản_phẩm phù_hợp giá tiền
3030,POS,4,màu đẹp ưng_ý trong áo_khoác mua
...,...,...,...
29887,POS,5,độ bền mới
5412,NEU,3,màu cam đỏ ổn nhưng xanh chất như áo_mưa còn c...
860,POS,4,sản_phẩm sale đẹp chất_lượng
15839,POS,4,chất_lượng sản_phẩm kém không đáng tiền không ...


In [ ]:
print(df_train.shape)
print(df_test.shape)

(25095, 3)
(6274, 3)


In [ ]:
df_train.to_csv('/content/drive/MyDrive/NLP Final Project/data_train.csv')
df_test.to_csv('/content/drive/MyDrive/NLP Final Project/data_test.csv')